## 8.2文本预处理
文本预处理的步骤：

1. 按行读入文本
2. 分词：将字符串拆分为词元（如单词或字符）
3. 建立词表：将拆分的词元映射到数字索引
5. 索引转换：将文本转换成数字索引，方便模型操作。

tokens:分词后的数据

vocab:词表，包含了每个词元对应的索引

corpus:文本对应的词元的索引集合

### 读取数据

In [1]:
import collections
from d2l import torch as d2l
import re

In [2]:
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt',
                                '090b5e7e70c295757f55df93cb0a180b9691891a')
def read_time_machine():
    with open(d2l.download('time_machine'), 'r') as f:
        lines = f.readlines()
        # 正则表达
        sentences = list()
        for line in lines:
            sentence = re.sub('[^A-Za-z]+',' ' ,line).strip().lower()
            sentences.append(sentence)
    return sentences
    # return [re.sub('[^A-Za-z]+',' ' ,line).strip().lower() for line in lines]

lines = read_time_machine()
lines[:10]

['the time machine by h g wells',
 '',
 '',
 '',
 '',
 'i',
 '',
 '',
 'the time traveller for so it will be convenient to speak of him',
 'was expounding a recondite matter to us his grey eyes shone and']

### 词元化，分词（Tokenization）

In [3]:
def tokenize(lines, token='word'):
    if token == 'word':
        return [line.split() for line in lines]
    elif token == 'char':
        return [list(line) for line in lines]
    else:
        print('ERROR: unknown token type: ' + token)

tokens = tokenize(lines)
tokens[:10]

[['the', 'time', 'machine', 'by', 'h', 'g', 'wells'],
 [],
 [],
 [],
 [],
 ['i'],
 [],
 [],
 ['the',
  'time',
  'traveller',
  'for',
  'so',
  'it',
  'will',
  'be',
  'convenient',
  'to',
  'speak',
  'of',
  'him'],
 ['was',
  'expounding',
  'a',
  'recondite',
  'matter',
  'to',
  'us',
  'his',
  'grey',
  'eyes',
  'shone',
  'and']]

In [4]:
## 字典数据集
class Vocab:
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        """
        文本词表
        
        Args:
            tokens: 
            min_freq: 统计的的最小词频
            reserved_tokens: 预留一些词（例如填充词元（“<pad>”）； 序列开始词元（“<bos>”）； 序列结束词元（“<eos>”））
        """
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        # 根据频次排序
        counter = count_corpus(tokens)
        self._token_freqs = sorted(counter.items() , key= lambda x:x[1],
                                  reverse=True)
        
        self.idx_to_token = ['unk'] + reserved_tokens
        self.token_to_idx = {token:idx
                            for idx, token in enumerate(self.idx_to_token)}
       
        for token, freq in self._token_freqs:
            if freq < min_freq:
                break
            if token not in self.token_to_idx:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1
                
    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk) # default =Non 不存在返回unk的索引
        return [self.__getitem__(token) for token in tokens]
    
    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]
    
    @property
    def unk(self):#未知词元的索引为0
        return 0
    @property
    def token_freqs(self):
        return self._token_freqs


# corpus 语料库
def count_corpus(tokens):
    '''
    计算 token 的频次
    '''
    # tokens 可能是 1D list or 2D list
    if len(tokens) == 0 or isinstance(tokens[0], list):
    # 将词元列表展平成一个列表
        tokens = [ token for line in tokens for token in line]
    return collections.Counter(tokens)

In [5]:
vocab = Vocab(tokens)
print(list(vocab.token_to_idx.items())[:10])

[('unk', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]


将文本转换成数字索引

In [6]:
for i in [0, 10]:
    print('words:', tokens[i])
    print('indices:', vocab[tokens[i]])

words: ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
indices: [1, 19, 50, 40, 2183, 2184, 400]
words: ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
indices: [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


### 整合所有功能
1. 为了简化后面章节中的训练，我们使用字符（而不是单词）实现文本词元化；

2. 时光机器数据集中的每个文本行不一定是一个句子或一个段落，还可能是一个单词，因此返回的corpus仅处理为单个列表，而不是使用多词元列表构成的一个列表。

In [7]:
def load_corpus_time_machine(max_tokens=-1):  #@save
    """
    返回时光机器数据集的词元索引列表和词表.
    Args:
        max_tokens:
    
    returns:
        corpus:词元索引列表
        vocab: 时光机器语料库的词表
    """
    lines = read_time_machine()
    tokens = tokenize(lines, 'char')
    vocab = Vocab(tokens)
    # Since each text line in the time machine dataset is not necessarily a
    # sentence or a paragraph, flatten all the text lines into a single list
    corpus = [vocab[token] for line in tokens for token in line]
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus, vocab

corpus, vocab = load_corpus_time_machine()
len(corpus), len(vocab)

(170580, 28)

In [8]:
vocab.token_freqs[:10],

([(' ', 29927),
  ('e', 17838),
  ('t', 13515),
  ('a', 11704),
  ('i', 10138),
  ('n', 9917),
  ('o', 9758),
  ('s', 8486),
  ('h', 8257),
  ('r', 7674)],)